# 🛰️ YOLOv10 Tomato Disease Object Detection
YOLOv10x is used to detect and localize tomato leaf diseases using bounding boxes.

In [ ]:
# Install Ultralytics which now includes YOLOv10 support
pip install ultralytics --upgrade

In [ ]:
# Check GPU and environment
import torch, os, yaml
print("CUDA Available:", torch.cuda.is_available())
!nvidia-smi

In [ ]:
# Set base directory
base_dir = '/workspace/datacentervision/Tomato-Village/Variant-c(Object Detection)'

# Define your class labels
class_names = [
    'Early_blight', 'Healthy', 'Late_blight', 'Leaf Miner',
    'Magnesium Deficiency', 'Nitrogen Deficiency',
    'Pottassium Deficiency', 'Spotted Wilt Virus'
]

# Create data.yaml
data_cfg = {
    'train': os.path.join(base_dir, 'train', 'images'),
    'val': os.path.join(base_dir, 'val', 'images'),
    'nc': len(class_names),
    'names': class_names
}

with open('data.yaml', 'w') as f:
    yaml.dump(data_cfg, f)

print("✔ data.yaml created:")
print(yaml.dump(data_cfg))

In [ ]:
from glob import glob
train_imgs = glob(os.path.join(data_cfg['train'], '*.*'))
val_imgs = glob(os.path.join(data_cfg['val'], '*.*'))
print(f"Train Images: {len(train_imgs)}")
print(f"Val Images:   {len(val_imgs)}")

In [ ]:
import xml.etree.ElementTree as ET
from PIL import Image
import matplotlib.pyplot as plt

def show_pascal(img_path, xml_path):
    img = Image.open(img_path)
    tree = ET.parse(xml_path)
    root = tree.getroot()
    plt.figure(figsize=(6,6)); plt.imshow(img); ax = plt.gca()
    for obj in root.findall('object'):
        name = obj.find('name').text
        bnd = obj.find('bndbox')
        x1, y1 = int(bnd.find('xmin').text), int(bnd.find('ymin').text)
        x2, y2 = int(bnd.find('xmax').text), int(bnd.find('ymax').text)
        ax.add_patch(plt.Rectangle((x1,y1), x2-x1, y2-y1, edgecolor='lime', fill=False, lw=2))
        ax.text(x1, y1-10, name, color='white', bbox=dict(facecolor='red', alpha=0.5))
    plt.axis('off'); plt.show()

# Example
sample_img = train_imgs[0]
sample_xml = os.path.join(base_dir, 'train', 'pascal_voc', os.path.basename(sample_img).replace('.jpg', '.xml'))
show_pascal(sample_img, sample_xml)


In [ ]:
import albumentations as A

aug_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5)
], bbox_params=A.BboxParams(format='yolo', label_fields=['labels']))

print("Augmentation pipeline ready (not applied during YOLO training)")


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov10x.pt')  # Use the best (largest) variant


In [ ]:
model.train(
    data='data.yaml',
    epochs=60,
    imgsz=640,
    batch=16,
    project='runs/train',
    name='yolov10-tomato',
    save=True,
    device=0  # or 'cuda'
)


In [ ]:
results = model.trainer.metrics
plt.plot(results['epoch'], results['box_loss'], label='Box Loss')
plt.plot(results['epoch'], results['cls_loss'], label='Class Loss')
plt.plot(results['epoch'], results['obj_loss'], label='Obj Loss')
plt.legend(); plt.title('Training Losses'); plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.grid(); plt.show()

In [ ]:
val_results = model.val()
print(f"Validation Results:\n mAP50: {val_results.box.map[0]:.4f} | mAP50-95: {val_results.box.map:.4f}")


In [ ]:
pred = model.predict(source=base_dir + "/val/images", conf=0.25, save=True)
print("✅ Inference complete. Annotated results saved.")


In [ ]:
model.export(format='onnx')
model.export(format='torchscript')
print("Model exported to ONNX and TorchScript formats.")
